Basic trigram model, Excercise 01 from karpathys lectures

In [6]:
import torch
from torch import nn

In [7]:
names = open('names.txt', 'r').read().splitlines()
names[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [8]:
# conversion functions
chars = sorted(list(set(''.join(names))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0 # special char denoting start/end
itos = {i:s for s,i in stoi.items()}

In [19]:

# Generate trigrams and count occurences
t = {}
for n in names:
    chs = ['.'] + list(n) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        trigram = (ch1, ch2, ch3)
        t[trigram] =  t.get(trigram,0) +1



In [24]:
N = torch.zeros((27,27,27), dtype=torch.int32)    
for (ch1, ch2, ch3), count in t.items():
    #print(f" {ch1}{ch2}{ch3}   count {count}")
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    N[ix1, ix2, ix3] =  count


Notes:
Add-one smoothing/Laplace-smoothing: avoid assigning a probability of 0 to unseen n-grams
keepdim: True --> output tensor has same dimensions as input tensor


In [42]:
#P(3rd char)/P(1+2nd char) --> trigram_count/count_all_trigrams_starting_with_same_bigram

P = (N+1).float() # add-one smoothing
P /= P.sum(2, keepdim=True) # Sum over 3rd dim and normalize
P

tensor([[[3.7037e-02, 3.7037e-02, 3.7037e-02,  ..., 3.7037e-02,
          3.7037e-02, 3.7037e-02],
         [2.2538e-04, 4.6879e-02, 4.3047e-02,  ..., 6.3106e-03,
          3.9216e-02, 3.4483e-02],
         [7.5019e-04, 1.2753e-01, 7.5019e-04,  ..., 7.5019e-04,
          3.7509e-03, 7.5019e-04],
         ...,
         [6.2112e-03, 3.6025e-01, 6.2112e-03,  ..., 1.2422e-02,
          1.1180e-01, 7.4534e-02],
         [1.7794e-03, 4.3950e-01, 1.7794e-03,  ..., 1.7794e-03,
          1.7794e-03, 5.3381e-03],
         [1.0460e-03, 4.7803e-01, 1.0460e-03,  ..., 1.0460e-03,
          9.6234e-02, 2.0921e-03]],

        [[3.7037e-02, 3.7037e-02, 3.7037e-02,  ..., 3.7037e-02,
          3.7037e-02, 3.7037e-02],
         [7.0326e-02, 1.7153e-03, 1.0292e-02,  ..., 1.7153e-03,
          3.6021e-02, 2.0583e-02],
         [6.5141e-02, 5.1056e-02, 3.6972e-02,  ..., 1.7606e-03,
          2.2887e-02, 1.7606e-03],
         ...,
         [5.7416e-02, 2.8708e-02, 4.7847e-03,  ..., 8.6124e-02,
          3.349

In [48]:
for i in range(5):
  out = ['.']
  while True:
    if len(out) < 2:
      ix1 = ix2 = stoi[out[-1]]
    else: 
      ix1 = stoi[out[-2]]
      ix2 = stoi[out[-1]]
    p = P[ix1,ix2] # p for next char 
    #p = torch.ones(27)/ 27.0  #sanity check that there is an effect actually --> make uniform distribution
    ix3 = torch.multinomial(p,num_samples=1,replacement=True).item() # Get actual number
    #print(f"{itos[ix1] }{itos[ix2]} next best char: {itos[ix3]}")
    if ix3 == stoi['.']:
      break
    out.append(itos[ix3])

  print(''.join(out[1:])) 


viufel
bett
dan
haea
gian
